In [1]:
"""
This notebook is to look at the data and to try to mimic the controller. The reason for this is that the laser does not
accept the commands for some reason when it is on. Must look at the pattern between the controller and the laser and at what intervals
they send each other messages, and which messages they send to each other.
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import binascii
import string


In [2]:
idle_main_menu = pd.read_csv('idle_main_menu.csv', delimiter = ",")
idle_main_menu

,Timestamp,Arbitration_id,dlc,data
0,1.757712e+09,0x18820810,5,020a016f09
1,1.757712e+09,0x18820810,3,070101
2,1.757712e+09,0x18820810,8,102045204f464620
3,1.757712e+09,0x18820810,4,00202000
4,1.757712e+09,0x18820810,5,0200007a20
...,...,...,...,...
3746,1.757712e+09,0x19000410,3,0a1fff
3747,1.757712e+09,0x19000410,3,021fff
3748,1.757712e+09,0x19000410,3,071fff
3749,1.757712e+09,0x19000410,3,101fff


In [3]:
def device_division(data):
    """
    Enter a dataset and it will return the data divided into the corresponding device.
    "0x18820810" --> The laser
    "0x19000410" --> The controller
    """
    
    laser = data[data['Arbitration_id'] == '0x18820810']
    controller = data[data['Arbitration_id'] == '0x19000410']
    return laser, controller
    

In [4]:
def different_values(Idle_data, command_data):
    laser_idle = Idle_data[Idle_data['Arbitration_id'] == '0x18820810']
    controller_idle = Idle_data[Idle_data['Arbitration_id'] == '0x19000410']
    
    laser = command_data[command_data['Arbitration_id'] == '0x18820810']
    controller = command_data[command_data['Arbitration_id'] == '0x19000410']

    Lidle_data_unique, Lidle_data_counts = np.unique(laser_idle['data'], return_counts = True)
    Cidle_data_unique, Cidle_data_counts = np.unique(controller_idle['data'], return_counts = True)

    Lcommand_data_unique, Lcommand_data_counts = np.unique(laser['data'], return_counts = True)
    Ccommand_data_unique, Ccommad_data_counts = np.unique(controller['data'], return_counts = True)

    # Now will add the different commands to a list to compare
    similar = []
    different = []

    for message in Ccommand_data_unique:
        if message in Cidle_data_unique:
            similar.append(message)
        else:
            different.append(message)
    # See how many times each one showed
    return similar, different
    

In [5]:
laser_idle, controller_idle = device_division(idle_main_menu)
laser_idle.head(10)

,Timestamp,Arbitration_id,dlc,data
0,1.757712e+09,0x18820810,5,020a016f09
1,1.757712e+09,0x18820810,3,070101
2,1.757712e+09,0x18820810,8,102045204f464620
3,1.757712e+09,0x18820810,4,00202000
4,1.757712e+09,0x18820810,5,0200007a20
5,1.757712e+09,0x18820810,5,02010c3b09
6,1.757712e+09,0x18820810,3,070101
8,1.757712e+09,0x18820810,8,10506d702020302e
9,1.757712e+09,0x18820810,4,00302500
10,1.757712e+09,0x18820810,5,0200007a20


In [6]:
controller_idle.head(10)

,Timestamp,Arbitration_id,dlc,data
7,1.757712e+09,0x19000410,3,021fff
27,1.757712e+09,0x19000410,3,071fff
28,1.757712e+09,0x19000410,3,101fff
29,1.757712e+09,0x19000410,3,101fff
30,1.757712e+09,0x19000410,3,021fff
31,1.757712e+09,0x19000410,3,071fff
32,1.757712e+09,0x19000410,3,021fff
33,1.757712e+09,0x19000410,3,101fff
34,1.757712e+09,0x19000410,3,101fff
35,1.757712e+09,0x19000410,3,021fff


In [7]:
laser_unique_data, laser_counts_data = np.unique(laser_idle['data'], return_counts = True)
laser_unique_data

array(['00202000', '0020312000', '00302500', '00f84300', '0200007a20',
       '02010c3b09', '0201163b09', '020a016f09', '023e0c3b09', '070101',
       '070202', '0a0000000901', '0a00003b0001', '0d00', '0d01', '101e00',
       '102045204f464620', '1020506b20202020', '10506d702020302e',
       '1074202020302e30', '12'], dtype=object)

In [8]:
controller_unique_data, controller_counts_data = np.unique(controller_idle['data'], return_counts = True)
controller_unique_data

array(['021fff', '071fff', '0a1fff', '0d1fff', '101fff', '121fff'],
      dtype=object)

In [9]:
# Now comparing the idle between the idle in main menu and idle in the burst mode menu
idle_burst_menu = pd.read_csv('can_log.csv', delimiter = ",")
idle_burst_menu.columns = ['Timestamp', 'Arbitration_id', 'dlc', 'data']


In [10]:
similar, different = different_values(idle_burst_menu, idle_main_menu)
print(similar)
print(different)
# It seems that they are similar, so it doesn't matter if the controller is on any screen.

['021fff', '071fff', '0a1fff', '0d1fff', '101fff', '121fff']
[]


In [11]:
# The next step is to analyse the data as the controller is starting up
laser_startup = pd.read_csv("laser_startup.csv", delimiter = ",")
laser_startup


,Timestamp,Arbitration_id,dlc,data
0,1.757956e+09,0x1803280a,4,00000000
1,1.757956e+09,0x3080a,4,00000000
2,1.757956e+09,0x18820810,1,01
3,1.757956e+09,0x19000410,3,010000
4,1.757956e+09,0x18820810,1,03
...,...,...,...,...
2066,1.757956e+09,0x19000410,3,0a1fff
2067,1.757956e+09,0x19000410,3,021fff
2068,1.757956e+09,0x19000410,3,071fff
2069,1.757956e+09,0x19000410,3,101fff


In [12]:
unique_devices, count_devices = np.unique(laser_startup['Arbitration_id'], return_counts = True)
print(unique_devices)
print(count_devices)
#It seems like there are now two more devices on the network

['0x1803280a' '0x18820810' '0x19000410' '0x3080a']
[   1 1035 1034    1]


In [13]:
def device_split(data):
    unique_devices = np.unique(data['Arbitration_id'])

    devices = []
    for device in unique_devices:
        devices.append(data[data['Arbitration_id'] == f'{device}'])

    return devices

In [14]:
Devices = device_split(laser_startup)
Devices

[      Timestamp Arbitration_id  dlc      data
 0  1.757956e+09     0x1803280a    4  00000000,
          Timestamp Arbitration_id  dlc              data
 2     1.757956e+09     0x18820810    1                01
 4     1.757956e+09     0x18820810    1                03
 5     1.757956e+09     0x18820810    5        020a016f09
 6     1.757956e+09     0x18820810    3            070101
 7     1.757956e+09     0x18820810    8  102045204f464620
 ...            ...            ...  ...               ...
 2041  1.757956e+09     0x18820810    6      0a0000000901
 2042  1.757956e+09     0x18820810    5        0200007a20
 2043  1.757956e+09     0x18820810    3            070202
 2044  1.757956e+09     0x18820810    3            101e00
 2070  1.757956e+09     0x18820810    1                12
 
 [1035 rows x 4 columns],
          Timestamp Arbitration_id  dlc    data
 3     1.757956e+09     0x19000410    3  010000
 34    1.757956e+09     0x19000410    3  030000
 35    1.757956e+09     0x19000410   

In [15]:
Device1 = Devices[0]
Device2 = Devices[1]
Device3 = Devices[2]
Device4 = Devices[3]

In [16]:

def hex_to_ascii(hexstr):
    """Convert hex string to ASCII, replace non-printables with '.' """
    try:
        hexstr = str(hexstr).strip()
        if hexstr.lower() in ("nan", "none", ""):
            return ""
        bytes_data = binascii.unhexlify(hexstr)
        return ''.join([chr(b) if chr(b) in string.printable else '.' for b in bytes_data])
    except Exception:
        return ""

def decode_can_csv(filename):
    # Load CSV as strings, ignore bad lines
    df = pd.read_csv(filename, dtype=str, header=0, on_bad_lines="skip")

    # Locate the data column
    possible_cols = [c for c in df.columns if "data" in c.lower()]
    if not possible_cols:
        raise ValueError("No 'data' column found in CSV")
    data_col = possible_cols[0]

    # Convert hex payloads to ASCII
    df["ascii"] = df[data_col].astype(str).apply(hex_to_ascii)

    # Print hex alongside ASCII
    decoded = []
    for raw_hex, ascii_text in zip(df[data_col], df["ascii"]):
        if raw_hex and raw_hex.lower() not in ("nan", "none", ""):
            decoded.append((raw_hex, ascii_text))

    return decoded

In [19]:
filename = "laser_startup.csv"  # change to your file
decoded = decode_can_csv(filename)

if decoded:
    print(f"Decoded {len(decoded)} frames:\n")
    for raw, ascii_txt in decoded:
        print(f"HEX: {raw:<20}  ASCII: {ascii_txt}")
else:
    print("No valid frames found.")

Decoded 2071 frames:

HEX: 00000000              ASCII: ....
HEX: 00000000              ASCII: ....
HEX: 01                    ASCII: .
HEX: 010000                ASCII: ...
HEX: 03                    ASCII: .
HEX: 020a016f09            ASCII: .
.o	
HEX: 070101                ASCII: ...
HEX: 102045204f464620      ASCII: . E OFF 
HEX: 00202000              ASCII: .  .
HEX: 0200007a20            ASCII: ...z 
HEX: 02010c3b09            ASCII: ..;	
HEX: 070101                ASCII: ...
HEX: 10506d702020302e      ASCII: .Pmp  0.
HEX: 00302500              ASCII: .0%.
HEX: 0200007a20            ASCII: ...z 
HEX: 023e0c3b09            ASCII: .>;	
HEX: 070101                ASCII: ...
HEX: 1074202020302e30      ASCII: .t   0.0
HEX: 00f84300              ASCII: ..C.
HEX: 0200007a20            ASCII: ...z 
HEX: 0201163b09            ASCII: ...;	
HEX: 070101                ASCII: ...
.EX: 0d00                  ASCII: 
HEX: 1020506b20202020      ASCII: . Pk    
HEX: 0020312000            ASCII: 